In [37]:
import numpy as np
import pandas as pd
import pickle
import wittgenstein as witt
import arff

In [27]:
df = pd.read_csv("table.csv")
df.dtypes
# cols = list(df.columns)
# cols.remove('objid')
# cols.remove('class')
# df = df.drop(df[df['class'] == 'QSO'].index)
df = df.drop(['fiberid','objid','specobjid','plate'],axis=1)

In [28]:
# X = df.drop('class',axis=1)
# X = X.drop('objid',axis=1).to_numpy()
# y = df['class'].to_numpy()
# y = np.array(list(map(lambda x: True if x == 'STAR' else False, y)))

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

Numerical to nominal

In [31]:
new_run = []
for elem in df['run'].to_numpy():
    new_run.append('id'+str(elem))
df['run'] = new_run

Test and train split and standarization

In [32]:
train,test = train_test_split(df,test_size=0.1)
numerical = ['ra','dec','u','g','r','i','z','rerun','camcol','field','redshift','mjd']

scaler = StandardScaler()
train[numerical] = scaler.fit_transform(train[numerical])

test[numerical] = scaler.transform(test[numerical])

In [38]:
train.to_csv('table_train.csv',index=False)
test.to_csv('table_test.csv',index=False)

arff.dump('table_train.arff'
      , train.values
      , relation='Train'
      , names=train.columns)

arff.dump('table_test.arff'
      , test.values
      , relation='Test'
      , names=test.columns)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
# Make a backup
backup = open("backup.obj","wb")
pickle.dump([X_train,X_test,y_train,y_test],backup)

In [7]:
# Get backup
backup = open("backup.obj","rb")
X_train, X_test, y_train, y_test = pickle.load(backup)
print(X_train)

[[ 1.29686902e+02  5.06189975e+01  1.80844000e+01 ...  4.45000000e+02
   5.18730000e+04  1.75000000e+02]
 [ 1.10070039e+02  1.21873469e+00  1.94259100e+01 ...  7.45000000e+03
   5.67220000e+04  7.05000000e+02]
 [ 1.64645630e+02  2.92387598e-01  1.84482900e+01 ...  2.76000000e+02
   5.19090000e+04  6.40000000e+02]
 ...
 [ 1.62836886e+02  6.59814568e-01  1.93594200e+01 ...  2.76000000e+02
   5.19090000e+04  4.80000000e+02]
 [ 1.89512933e+02 -1.72948791e+00  1.66057400e+01 ...  3.35000000e+02
   5.20000000e+04  4.69000000e+02]
 [ 1.52424573e+02  1.23747967e+00  1.95556300e+01 ...  5.02000000e+02
   5.19570000e+04  2.14000000e+02]]


Classifiers

In [8]:
# Grid Search
def print_best(clf,X_train, y_train):
    clf.fit(X_train, y_train)
    best_arg = np.argmax(clf.cv_results_['mean_test_score'])
    print('Best parameters: ', clf.cv_results_['params'][best_arg])
    print('Test score: ',clf.cv_results_['mean_test_score'][best_arg])
    print('Mean fit time: ',clf.cv_results_['mean_fit_time'][best_arg])
    print('Mean score time: ',clf.cv_results_['mean_score_time'][best_arg])
    return clf

# KNN
def knn_best(X_train,y_train,
             neighbors=[5,10,20,50], distance=['euclidean','manhattan','minkowski','chebyshev'],weights=['uniform','distance']):
    knn = KNeighborsClassifier()
    params = {'n_neighbors':neighbors,'metric':distance,'weights':weights}
    clf = GridSearchCV(estimator=knn,param_grid=params)
    return print_best(clf,X_train, y_train)

# Neural networks
def nn_best(X_train,y_train,
             num_hid_layers=[1,2,3], neur_per_layer=[8,16,32],activation=['relu','logistic']):
    hidden_layers = []
    for layers in num_hid_layers:
        for neur in neur_per_layer:
            hidden_layers.append((neur,)*layers)
    
    nn = MLPClassifier()
    params = {'hidden_layer_sizes':hidden_layers,'activation':activation}
    clf = GridSearchCV(estimator=nn,param_grid=params)
    return print_best(clf, X_train, y_train)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
print('KNN')
knn_best(X_train,y_train)
print('Neural networks:')
clf = nn_best(X_train,y_train)

KNN
Best parameters:  {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Test score:  0.9118666666666666
Mean fit time:  0.004069900512695313
Mean score time:  0.18379549980163573
Neural networks:


/home/dcabornero/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/dcabornero/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/dcabornero/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/dcabornero/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/dcabornero/.local/lib/python3.

Best parameters:  {'activation': 'relu', 'hidden_layer_sizes': (32, 32)}
Test score:  0.984
Mean fit time:  4.147333335876465
Mean score time:  0.003945112228393555


/home/dcabornero/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [96]:
# Classifiers
def neighbors(X_train,X_test,y_train,y_test):
    clf = KNeighborsClassifier(n_neighbors=50,metric='euclidean',weights='uniform')
    clf.fit(X_train,y_train)
    print(clf.score(X_test,y_test))
    return clf

def nn(X_train,X_test,y_train,y_test):
    clf = MLPClassifier()
    clf.fit(X_train,y_train)
    clf.score(X_test,y_test)
    return clf

def svm(X_train,X_test,y_train,y_test):
    clf = SVC(kernel='sigmoid')
    clf.fit(X_train,y_train)
    clf.score(X_test,y_test)
    return clf

def clas_tree(X_train,X_test,y_train,y_test):
    clf = DecisionTreeClassifier(criterion='gini')
    clf.fit(X_train,y_train)
    clf.score(X_test,y_test)
    return clf

def ripper(X_train,X_test,y_train,y_test):
    clf = witt.RIPPER(verbosity=5)
    clf.fit(X_train,y_train)
    clf.score(X_test,y_test)
    return clf

# Raw data

In [25]:
train = pd.read_csv('table_train.csv')
test = pd.read_csv('table_test.csv')
df = pd.concat(train,test,

#OneHotEncoder preprocessing
one = OneHotEncoder()
one.fit(train[discretize])
oneHotTrain = one.transform(train[discretize]).toarray()
oneHotTest = one.transform(test[discretize]).toarray()
X_train = train.drop(['class']+discretize,axis=1).to_numpy()
X_train = np.concatenate(X_train,oneHotTrain,axis=1)
#  = X.drop('objid',axis=1).to_numpy()
# y = df['class'].to_numpy()

           ra       dec         u         g         r         i         z  \
0    1.694284  1.747444 -0.544447 -0.254843  0.200757  0.397372  0.552310   
1    0.973498 -0.571589  0.414481  0.222809  0.183386  0.217890  0.262468   
2   -0.280463 -0.578670 -0.266667  0.127993  0.273936  0.359213  0.431532   
3    0.123626 -0.610801  0.851036  0.985996  1.010093  1.032673  1.081217   
4   -0.049138 -0.624250  0.595187  0.603023  0.532292  0.551096  0.560531   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.648053 -0.581427 -0.011132  0.130099  0.210983  0.269954  0.318231   
996 -0.305730 -0.560126 -0.564120 -1.213192 -1.270337 -1.181582 -1.099088   
997 -0.495163  1.857544 -0.501345 -0.587685 -0.674473 -0.752825 -0.795825   
998 -0.771554  1.670426 -1.304589 -0.872324 -0.541736 -0.384457 -0.269534   
999  0.285199  2.062416  0.883414  0.471272  0.027230 -0.139441 -0.235858   

        run  rerun    camcol     field      specobjid   class  redshift  \


ValueError: Found unknown categories ['id837', 'id895', 'id983', 'id992', 'id938', 'id677', 'id850', 'id833', 'id883', 'id866', 'id711', 'id686', 'id697', 'id665', 'id800', 'id988', 'id773', 'id698', 'id791', 'id899'] in column 0 during transform

In [106]:
X_train, y_train = train.drop('class',axis=1).to_numpy(), train['class'].to_numpy()
X_test, y_test = test.drop('class',axis=1).to_numpy(), test['class'].to_numpy()

one = OneHotEncoder()
one.fit(train[discretize])
train2 = train.copy()
print(one.transform(train2[discretize]).toarray())

# nn(X_train,X_test,y_train,y_test)

[12. 20. 19. ...  3.  1.  5.]


In [65]:
# train, test = train_test_split(df, random_state=42)
# clf = witt.RIPPER(verbosity=5)
# clf.fit(train)
# clf.score(X_test,y_test)

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
5944,1.237650e+18,182.420082,-0.848184,17.53683,16.70986,16.39093,16.26922,16.24099,756,301,1,445,3.722300e+18,STAR,0.000549,3306,54922,289
2391,1.237650e+18,170.049399,-0.446933,19.23571,17.86737,17.17782,16.78271,16.59521,756,301,2,362,3.153220e+17,GALAXY,0.103194,280,51612,256
3061,1.237650e+18,171.774166,-0.476422,18.61988,17.42074,16.93513,16.55151,16.61857,756,301,2,374,3.640160e+18,STAR,0.000460,3233,54891,472
6481,1.237650e+18,246.314102,-0.702445,19.47715,18.09879,17.35394,16.90334,16.60784,752,301,2,686,4.098640e+17,GALAXY,0.097080,364,52000,133
9998,1.237650e+18,131.477151,51.753068,18.88287,17.91068,17.53152,17.36284,17.13988,1345,301,3,163,5.033400e+17,GALAXY,0.014019,447,51877,229
